# Input Processing

In [1]:
import time

data = open("data/19.txt", "r").read()

In [2]:
from dataclasses import dataclass


@dataclass
class Blueprint:
    id: int
    ore_robot_ore_cost: int
    clay_robot_ore_cost: int
    obsidian_robot_ore_cost: int
    obsidian_robot_clay_cost: int
    geode_robot_ore_cost: int
    geode_robot_obsidian_cost: int

In [3]:
import re

pattern = re.compile(r"Blueprint (\d+):\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+)\D+(\d+) \w+\.")

blueprints: list[Blueprint] = []
for match in pattern.findall(data):
    blueprints.append(Blueprint(*(int(group) for group in match)))

# Part 1

In [4]:
def add_s(test_num: int):
    if test_num != 0:
        return "s"
    return ""

In [5]:
def rm_s(test_num: int):
    if test_num != 0:
        return ""
    return "s"

In [6]:
import enum


class PathMode(enum.Enum):
    TRY_MAKE = enum.auto()
    TRY_COLLECT = enum.auto()

In [7]:
def add_tuple(a: tuple[int, ...], b: tuple[int, ...]):
    assert len(a) == len(b)
    return tuple(a[i] + b[i] for i in range(len(a)))

In [8]:
def process_blueprint(test_blueprint: Blueprint, max_time=24, debug=False):
    stack: list[tuple[
        PathMode,
        tuple[int, int, int, int],
        tuple[int, int, int, int],
        tuple[int, int, int, int],
        int, str
    ]] = [(
        PathMode.TRY_MAKE,  # mode
        (1, 0, 0, 0),  # robots: ore, clay, obsidian, geode
        (0, 0, 0, 0),  # inventory: ore, clay, obsidian, geode
        (0, 0, 0, 0),  # future_robots: ore, clay, obsidian, geode
        1, ""  # timer, debug_string
    )]
    max_geode = -1
    max_collect = None

    top_geodes: list[int] = [0] * max_time

    while len(stack) > 0:
        hand = stack.pop() # DFS, need to perform greedy and prune
        mode, robots, inventory, future_robots, timer, debug_string = hand

        if timer > max_time:
            if inventory[3] > max_geode:
                max_geode = inventory[3]
                max_collect = hand
            continue

        match mode:
            case PathMode.TRY_MAKE:
                if debug:
                    debug_string += f"== Minute {timer} ==\n"
                # Factory can only make one robot at a time
                ## Don't build any robots
                stack.append((
                        PathMode.TRY_COLLECT,
                        robots, inventory, future_robots,
                        timer, debug_string if debug else ""))
                ## Try build ore robot
                if inventory[0] >= test_blueprint.ore_robot_ore_cost and robots[0] < max(test_blueprint.ore_robot_ore_cost, test_blueprint.clay_robot_ore_cost, test_blueprint.obsidian_robot_ore_cost, test_blueprint.geode_robot_ore_cost):
                    stack.append((
                        PathMode.TRY_COLLECT,
                        robots,
                        add_tuple(inventory, (-1 * test_blueprint.ore_robot_ore_cost, 0, 0, 0)),
                        (1, 0, 0, 0),
                        timer, (debug_string + f"Spend {test_blueprint.ore_robot_ore_cost} ore to start building a ore-collecting robot.\n") if debug else ""))
                ## Try build clay robot
                if inventory[0] >= test_blueprint.clay_robot_ore_cost and robots[1] < test_blueprint.obsidian_robot_clay_cost:
                    stack.append((
                        PathMode.TRY_COLLECT,
                        robots,
                        add_tuple(inventory, (-1 * test_blueprint.clay_robot_ore_cost, 0, 0, 0)),
                        (0, 1, 0, 0),
                        timer, (debug_string + f"Spend {test_blueprint.clay_robot_ore_cost} ore to start building a clay-collecting robot.\n") if debug else ""))
                ## Try build obsidian robot
                if inventory[0] >= test_blueprint.obsidian_robot_ore_cost and inventory[
                    1] >= test_blueprint.obsidian_robot_clay_cost and robots[2] < test_blueprint.geode_robot_obsidian_cost:
                    stack.append((
                        PathMode.TRY_COLLECT,
                        robots,
                        add_tuple(inventory, (
                            -1 * test_blueprint.obsidian_robot_ore_cost, -1 * test_blueprint.obsidian_robot_clay_cost,
                            0,
                            0)),
                        (0, 0, 1, 0),
                        timer, (debug_string + f"Spend {test_blueprint.obsidian_robot_ore_cost} ore and {test_blueprint.obsidian_robot_clay_cost} clay to start building an obsidian-collecting robot.\n") if debug else ""))
                ## Try build geode robot
                if inventory[0] >= test_blueprint.geode_robot_ore_cost and inventory[
                    2] >= test_blueprint.geode_robot_obsidian_cost:
                    stack.append((
                        PathMode.TRY_COLLECT,
                        robots,
                        add_tuple(inventory, (
                            -1 * test_blueprint.geode_robot_ore_cost, 0, -1 * test_blueprint.geode_robot_obsidian_cost,
                            0)),
                        (0, 0, 0, 1),
                        timer, (debug_string + f"Spend {test_blueprint.geode_robot_ore_cost} ore and {test_blueprint.geode_robot_obsidian_cost} obsidian to start building an obsidian-collecting robot.\n") if debug else ""))

            case PathMode.TRY_COLLECT:
                # Collect
                inventory = add_tuple(inventory, robots)

                if debug:
                    if robots[0]:
                        debug_string += f"{robots[0]} ore-collecting robot{add_s(robots[0])} collect{rm_s(robots[0])} {robots[0]} ore; you now have {inventory[0]} ore.\n"
                    if robots[1]:
                        debug_string += f"{robots[1]} clay-collecting robot{add_s(robots[1])} collect{rm_s(robots[1])} {robots[1]} clay; you now have {inventory[1]} clay.\n"
                    if robots[2]:
                        debug_string += f"{robots[2]} obsidian-collecting robot{add_s(robots[2])} collect{rm_s(robots[2])} {robots[2]} obsidian; you now have {inventory[2]} obsidian.\n"
                    if robots[3]:
                        debug_string += f"{robots[3]} geode-collecting robot{add_s(robots[3])} collect{rm_s(robots[3])} {robots[3]} ore; you now have {inventory[3]} open geode{add_s(inventory[3])}.\n"

                # Apply top_geodes
                top_geodes[timer - 1] = max(inventory[3], top_geodes[timer - 1])
                # Prune if underperforming
                if inventory[3] < top_geodes[timer - 1]:
                    continue

                # Apply future_robots
                robots = add_tuple(robots, future_robots)
                if debug:
                    if future_robots[0]:
                        debug_string += f"The new ore-collecting robot is ready; you now have {robots[0]} of them.\n"
                    if future_robots[1]:
                        debug_string += f"The new clay-collecting robot is ready; you now have {robots[1]} of them.\n"
                    if future_robots[2]:
                        debug_string += f"The new obsidian-collecting robot is ready; you now have {robots[2]} of them.\n"
                    if future_robots[3]:
                        debug_string += f"The new geode-collecting robot is ready; you now have {robots[3]} of them.\n"

                stack.append((
                    PathMode.TRY_MAKE,
                    robots, inventory, (0, 0, 0, 0),
                    timer + 1, (debug_string + "\n") if debug else ""
                ))
    return max_collect

In [9]:
# DEBUG: 110.81703440000001
# NORMAL: 51.20457280000028
# start = time.perf_counter()
# sum(process_blueprint(blueprint, debug=True)[2][3] * blueprint.id for blueprint in blueprints)
# print(f"DEBUG: {time.perf_counter() - start}")
# start = time.perf_counter()
# sum(process_blueprint(blueprint)[2][3] * blueprint.id for blueprint in blueprints)
# print(f"NORMAL: {time.perf_counter() - start}")

DEBUG: 110.81703440000001
NORMAL: 51.20457280000028


In [10]:
sum(process_blueprint(blueprint)[2][3] * blueprint.id for blueprint in blueprints)